INGESTA PERU

QUERY ORIGEN

In [ ]:
WITH InitialSales AS (
    SELECT  
        a.cIDCompany AS Company_DwID,
        a.cIDInvoice AS Invoice_DwID,
        a.cSerie AS Serie,
        a.cForm AS Form,
        a.cIDCustomer AS Customer_DwID,
        a.cIDBranchInvoice AS BranchDwID,
        a.cIDTerritory AS Territory_DwID,
        a.dEmission AS Emission,
        a.xChannelDistribution AS ChannelDistribution,
        a.xInvoiceCancelReason AS InvoiceCancelReason,
        b.cIDOrder AS Order_DwID,
        a.xInvoiceType AS InvoiceType,
        e.cIDProduct AS Product_DwID,
        e.nAmount AS Amount,
        e.nUnitValue AS UnitValue,
        e.nTotalValue AS TotalValue,
        e.MC1LastUpdate,
        e.nFreegoods AS Freegoods,
        g.nSeq AS Seq,
        g.nValue AS [Value],
        g.cIDPricing AS Pricing_DwID,
        g.cIDStep AS Step_DwID,
        f.xIDReasonReturnStock AS ReasonReturnStock_DwID,
        h.lCalculated AS Calculated,
        h.cIDPricingFormationRule AS PricingFormationRule_DwID
    FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_Invoice a
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderInvoice b ON
        a.cIDCompany = b.cIDCompany AND
        a.cIDInvoice = b.cIDInvoice AND
        a.cForm = b.cForm AND
        a.cSerie = b.cSerie AND
        a.cIDBranchInvoice = b.cIDBranchInvoice
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceItem e ON
        e.cIDCompany = a.cIDCompany AND
        e.cIDInvoice = a.cIDInvoice AND
        e.cForm = a.cForm AND
        e.cSerie = a.cSerie AND
        e.cIDBranchInvoice = a.cIDBranchInvoice
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceItemDetail g ON
        g.cIDCompany = e.cIDCompany AND
        g.cIDInvoice = e.cIDInvoice AND
        g.cForm = e.cForm AND
        g.cSerie = e.cSerie AND
        g.cIDBranchInvoice = e.cIDBranchInvoice AND
        g.nSeq = e.nSeq AND
        g.cIDProduct = e.cIDProduct AND
        g.cIDStep IN ('00100', '00200', '00400', '00600', '00700', '00800', '00900', '01100', '01200', '01300', '00150')
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderExt f ON
        f.cIDOrder = b.cIDOrder AND
        f.cIDCompany = b.cIDCompany
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceRequestEmites m ON
        a.cIDCompany = m.cIDCompany AND
        a.cIDInvoice = m.cIDInvoice AND
        a.cForm = m.cForm AND
        a.cSerie = m.cSerie AND
        a.cIDBranchInvoice = m.cIDBranchInvoice AND
        UPPER(m.xInvoiceRequestStatus) = 'SUCCEEDED'
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_PricingFormationRule h ON
        g.cIDStep = h.cIDStep AND
        g.cIDPricing = h.cIDPricing AND
        g.cIDCompany = h.cIDCompany AND
        UPPER(h.cIDPricingFormationRule) != 'UNITVALUECALCULATED'
    WHERE
        a.mc1Enabled = 1 AND
        b.mc1Enabled = 1 AND
        e.mc1Enabled = 1 AND
        g.mc1Enabled = 1 AND
        f.mc1Enabled = 1 AND
        m.mc1Enabled = 1 AND
        h.mc1Enabled = 1 AND
        a.xInvoiceType NOT IN ('WTM002', 'WTM005', 'ZOGRD', 'ZOGR') AND
        CONVERT(DATE, a.dEmission) BETWEEN '2025-01-02' AND '2025-01-07'
),
ValidOrders AS (
    SELECT DISTINCT
        b.cIDOrder,
        b.cIDProduct,
        a.cIDBranchInvoice
    FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderInvoice a
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderItem b ON
        a.cIDOrder = b.cIDOrder AND
        a.cIDCompany = b.cIDCompany AND
        a.cIDBranchInvoice = b.cIDBranchInvoice
    WHERE
        a.mc1Enabled = 1 AND
        b.mc1Enabled = 1 AND
        CONVERT(DATE, a.mc1LastUpdate) BETWEEN '2025-01-02' AND '2025-01-07'
),
ValidSales AS (
    SELECT 
        a.*--, 
        --b.cIDOrder
    FROM InitialSales a
    INNER JOIN ValidOrders b ON
        a.Order_DwID = b.cIDOrder AND
        a.Product_DwID = b.cIDProduct 
        and b.cIDOrder is not null
        --AND a.BranchDwID = b.cIDBranchInvoice
),
FirstDeduplicates AS (
    SELECT
        *
    FROM (
        SELECT
            a.*,
            ROW_NUMBER() OVER (
                PARTITION BY a.BranchDwID, a.ChannelDistribution, a.Company_DwID, a.Customer_DwID, a.Emission, a.Form, a.FreeGoods,
                a.Invoice_DwID, a.InvoiceCancelReason, a.InvoiceType, a.Pricing_DwID, a.Product_DwID, a.Seq, a.Serie, a.Step_DwID, a.Territory_DwID
                ORDER BY a.MC1LastUpdate DESC, a.ReasonReturnStock_DwID DESC
            ) AS rn
        FROM ValidSales a
    ) sub
    WHERE rn = 1
),
Transformations AS (
    SELECT
        a.*,
        CASE WHEN a.Step_DwID = '00100' THEN [Value] ELSE 0 END AS Valor_Bruto,
        CASE WHEN a.Step_DwID = '00400' THEN [Value] ELSE 0 END AS Promocion_ML,
        CASE WHEN a.Step_DwID = '00600' THEN [Value] ELSE 0 END AS Descuento_ML,
        CASE WHEN a.Step_DwID = '00150' THEN [Value] ELSE 0 END AS Escalafc_ML,
        CASE 
            WHEN a.Step_DwID = '00200' AND a.Freegoods = 0 THEN a.[Value] 
            WHEN a.Step_DwID = '00100' AND a.Freegoods = 1 THEN a.[Value] 
            ELSE 0
        END AS Facdsl_ML,
        CASE WHEN a.Step_DwID = '01300' THEN [Value] ELSE 0 END AS Impuesto1,
        CASE WHEN a.Step_DwID = '01100' THEN [Value] ELSE 0 END AS Impuesto2,
        CASE WHEN a.Step_DwID = '01400' THEN [Value] ELSE 0 END AS Venta_Impuestos
    FROM FirstDeduplicates a
    --WHERE a.cIDOrder IS NOT NULL
),
GroupedSales AS (
    SELECT
    	Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2,
        SUM(Escalafc_ML) AS Escalafc_ML
    FROM Transformations
    GROUP BY
    	Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue
),
GroupedSales2 AS (
      SELECT
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Amount) AS Amount,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2,
        SUM(Escalafc_ML) AS Escalafc_ML
    FROM GroupedSales
    GROUP BY
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        Territory_DwID,
        UnitValue
),
InvoiceItemDetailJoin AS (
    SELECT 
        a.cIDCompany AS Company_DwID,
        a.cIDBranchInvoice AS BranchInvoice_DwID,
        a.cSerie AS Serie,
        a.cIDInvoice AS Invoice_DwID,
        a.cForm AS Form,
        a.cIDProduct AS Product_DwID,
        CONVERT(VARCHAR(16), a.MC1LastUpdate, 120) AS MC1LastUpdate, -- yyyy-MM-dd hh:mm
        c.nFreegoods AS FreeGoods,
        a.nValue AS nValue
    FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceItemDetail a
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_Invoice b ON
        a.cIDBranchInvoice = b.cIDBranchInvoice AND
        a.cSerie = b.cSerie AND
        a.cIDInvoice = b.cIDInvoice AND
        a.cForm = b.cForm
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceItem c ON
        a.cIDBranchInvoice = c.cIDBranchInvoice AND
        a.cSerie = c.cSerie AND
        a.cIDInvoice = c.cIDInvoice AND
        a.cForm = c.cForm AND
        a.cIDProduct = c.cIDProduct AND
        a.nSeq = c.nSeq
    WHERE
        a.cIDStep = '00700' AND
        a.MC1Enabled = 1 AND
        b.MC1Enabled = 1 AND
        c.MC1Enabled = 1 AND
        CONVERT(DATE, b.dEmission) BETWEEN '2025-01-02' AND '2025-01-07' AND
        b.xInvoiceType NOT IN ('WTM002', 'WTM005', 'ZOGRD', 'ZOGR')
),
InvoiceItemDetailDeduplicated AS (
    SELECT DISTINCT 
        *
    FROM InvoiceItemDetailJoin
),
InvoiceDetailGenerated AS (
    SELECT 
        Company_DwID,
        BranchInvoice_DwID,
        Serie,
        Invoice_DwID,
        Form,
        Product_DwID,
        MC1LastUpdate,
        FreeGoods,
        CASE WHEN FreeGoods = 1 THEN 0 ELSE nValue END AS nValue
    FROM InvoiceItemDetailDeduplicated
),
InvoiceDetailGrouped AS (
    SELECT
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie,
        SUM(nValue) AS facdet
    FROM InvoiceDetailGenerated
    GROUP BY 
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie
),
FinalJoin AS (
    SELECT 
        a.*,
        b.facdet
    FROM GroupedSales2 a
    INNER JOIN InvoiceDetailGrouped b ON
        a.Company_DwID = b.Company_DwID AND
        a.BranchDwID = b.BranchInvoice_DwID AND
        a.Product_DwID = b.Product_DwID AND
        a.Serie = b.Serie AND
        a.Form = b.Form AND
        a.Invoice_DwID = b.Invoice_DwID
),
AddColumns AS (
    SELECT 
        a.Amount,
        a.BranchDwID,
        a.ChannelDistribution,
        a.Company_DwID,
        a.Customer_DwID,
        a.Descuento_ML,
        a.Escalafc_ML,
        a.Facdsl_ML,
        --a.Freegoods,
        a.InvoiceCancelReason,
        a.Invoice_DwID,
        a.InvoiceType,
        a.Order_DwID,
        a.Promocion_ML,
        a.ReasonReturnStock_DwID,
        --a.Seq,
        a.Serie,
        a.Territory_DwID,
        a.UnitValue,
        a.Valor_Bruto,
        a.Venta_Impuestos,
        a.Impuesto1,
        a.Impuesto2,
        CONCAT(a.Serie, '-', a.Invoice_DwID) AS Numero,
        a.ChannelDistribution AS Cod_Linea_ruta,
        a.Company_DwID AS Pais,
        a.InvoiceType AS AG19ID,
        a.Customer_DwID AS Codigo_Cliente,
        a.BranchDwID AS Codigo_Agencia,
        a.Territory_DwID AS Codigo_Ruta,
        a.Product_DwID AS Codigo_Producto,
        'FALSE' AS FACCON,
        CASE
            WHEN a.InvoiceType IN ('ZBON', 'ZREX', 'ZOREX') THEN 0 - facdet
            ELSE facdet
        END AS facdet,
        CASE 
            WHEN a.InvoiceType IN ('ZREX', 'ZOREX') THEN a.ReasonReturnStock_DwID
            WHEN a.InvoiceType NOT IN ('ZREX', 'ZOREX') THEN a.InvoiceCancelReason
            ELSE 'REVISAR'
        END AS Id_Motivo,
        CONVERT(VARCHAR(10), CAST(a.Emission AS DATE), 120) AS Fecha,
        REPLACE(a.Territory_DwID, a.BranchDwID, '') AS Territory_DwID_Short
    FROM FinalJoin a
),
JoinWithSubchannel AS (
    SELECT *
    FROM (
        SELECT 
            a.*,
            b.cidbranchinvoice,
            ROW_NUMBER() OVER (PARTITION BY b.cidbranchinvoice, b.cidcustomer ORDER BY a.mc1Enabled DESC, a.mc1LastUpdate DESC) AS rn
        FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_CustomerDivisionSectorExt a
        LEFT JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_CustomerDivisionSector b ON
            a.cidcustomer = b.cidcustomer
        WHERE 
            b.cidbranchinvoice IS NOT NULL
    ) sub
    WHERE rn = 1
),
FinalTablaSubchannel AS (
    SELECT 
        a.*,
        b.xSubChannelDistribution
    FROM AddColumns a
    LEFT JOIN JoinWithSubchannel b ON
        a.Codigo_Cliente = b.cidCustomer AND
        a.Codigo_Agencia = b.cidbranchinvoice
),
Transformations2 AS (
    SELECT
        a.*,
        -- Renombrar columnas
        CONVERT(VARCHAR(10), a.Fecha, 120) AS Fecha_vta,
        a.Amount AS cantidad,
        a.UnitValue AS precio,
        a.Descuento_ML AS descuento,
        a.Promocion_ML AS Promocion,
        a.Facdsl_ML AS [FACDSL $],
        --a.facdet AS FACDET,
        a.Invoice_DwID AS no_facturas,
        a.Codigo_Cliente AS CLICOD,
        -- Ajustes condicionales
        CASE
            WHEN a.Cod_Linea_ruta <> a.xSubChannelDistribution AND a.xSubChannelDistribution IS NOT NULL THEN a.xSubChannelDistribution
            ELSE a.Cod_Linea_ruta
        END AS Cod_Linea_ruta_Adjusted,
        CASE
            WHEN (a.Id_Motivo IS NULL OR a.Id_Motivo = '') AND a.InvoiceCancelReason <> '' THEN a.InvoiceCancelReason
            ELSE a.Id_Motivo
        END AS Id_Motivo_Adjusted
    FROM FinalTablaSubchannel a
),
Transformations3 AS (
    SELECT
        a.*,
        -- Ajustar columnas financieras basadas en AG19ID
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) IN ('ZBON', 'ZOREX', 'ZREX') THEN 0 - a.[FACDSL $]
            ELSE a.[FACDSL $]
        END AS [FACDSL_Adjusted],
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) IN ('ZBON', 'ZOREX', 'ZREX') THEN 0 - a.descuento
            ELSE a.descuento
        END AS descuento_Adjusted,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) IN ('ZBON', 'ZOREX', 'ZREX') THEN 0 - a.Promocion
            ELSE a.Promocion
        END AS Promocion_Adjusted,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) IN ('ZBON', 'ZOREX', 'ZREX') THEN 0 - a.Escalafc_ML
            ELSE a.Escalafc_ML
        END AS Escalafc_ML_Adjusted
    FROM Transformations2 a
),
ReturnReason as (
    SELECT 
        cDescription as Description,
        cIDDomainType,
        cDomainType
    from CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_GeneralDescription
    where cDomainType = 'xInvoiceCancelReason'
    and lEnabled = 1
),
JoinReturnReason as (
    SELECT 
        a.*,
        b.Description
    from Transformations3 a
    left join ReturnReason b
    on a.InvoiceCancelReason = b.cIDDomainType
),
Motivos as (
    SELECT 
        cDescription as Description,
        cIDDomainType,
        cDomainType
    from CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_GeneralDescription
    where cDomainType = 'xIDReasonReturnStock'
    and lEnabled = 1
),
JoinMotivos as (
    SELECT 
        a.*,
        b.Description as Description2
    from JoinReturnReason a
    left join Motivos b
    on a.Id_Motivo = b.cIDDomainType
),
Transformations4 as (
    select
        a.*,
        case when a.Description is null and a.Description2 != '' then a.Description2 else a.Description end as factura_cliente
    from JoinMotivos a
    where Codigo_Ruta <> '0A'
),
prejoin as (
	SELECT 
        a.BranchDwID codigo_agencia,
        a.Customer_DwID CLICOD,
        a.Fecha_vta,
        a.Invoice_DwID no_factura,
        a.InvoiceCancelReason,
        a.AG19ID,
        a.codigo_producto,
        a.Territory_DwID_Short codigo_ruta,
        a.UnitValue precio,
        null dif_precio,
        a.FACDSL_Adjusted FACDSL$ ,
        a.Promocion_Adjusted promocion,
        a.descuento_Adjusted descuento,
        a.cantidad,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.numero,
        a.FACCON,
        a.[FACDSL $] FACDSL,
        a.FACDET,
        a.Id_Motivo_Adjusted Id_Motivo,
        a.factura_cliente,
        a.ChannelDistribution cod_linea_ruta
    FROM Transformations4 a	
),
InitialSalesRechazos as (
    SELECT  
    a.cIDCompany Company_DwID
    ,a.cIDInvoice Invoice_DwID
    ,a.cSerie Serie
    ,a.cForm Form
    ,a.cIDCustomer Customer_DwID 
    ,a.cIDBranchInvoice BranchDwID
    ,a.cIDTerritory Territory_DwID
    ,a.dEmission Emission
    ,a.xChannelDistribution ChannelDistribution 
    ,a.xInvoiceCancelReason InvoiceCancelReason
    ,b.cidOrder Order_DwID
    ,a.xInvoiceType InvoiceType 
    ,e.cIDProduct Product_DwID
    ,e.nAmount Amount
    ,e.nUnitValue UnitValue
    ,e.nTotalValue TotalValue
    ,e.MC1LastUpdate
    ,e.nFreegoods Freegoods
    ,g.nSeq Seq
    ,g.nValue Value
    ,g.cIDPricing Pricing_DwID
    ,g.cIDStep Step_DwID
    ,f.xIDReasonReturnStock ReasonReturnStock_DwID
    ,h.lCalculated Calculated
    ,h.cIDPricingFormationRule PricingFormationRule_DwID
    ,CASE 
    WHEN i.lPartialDelivery = '1' and l.nAmount!=0 THEN 'RECHAZO PARCIAL'
        ELSE 'RECHAZO TOTAL'  end Tipo_Rechazo
    ,CASE 
    WHEN i.lPartialDelivery = '1' and l.nAmount!=0 THEN '1'
        ELSE '2'  end Tipo_Rechazo_Id
    ,k.cIDInvoice InvoiceFac_DwID
    ,k.cSerie SerieFac
    ,isnull(l.nAmount,0) as AmountFac
    from CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_Invoice a
    --################# order invoice #########################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderInvoice 
    ) b 
    on a.cIDCompany = b.cIDCompany
    and a.cIDInvoice = b.cIDInvoice
    and a.cForm = b.cform
    and a.cSerie = b.cSerie
    and a.cIDBranchInvoice =b.cIDBranchInvoice
    AND a.mc1Enabled = 1
    and b.mc1enabled = 1
    --################## invoice item #####################
    inner join (select *
    FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceItem
    ) e 
    on e.cIDCompany = a.cIDCompany 
    and e.cIDInvoice = a.cIDInvoice 
    and e.cForm = a.cform
    and e.cSerie = a.cSerie
    AND e.cIDBranchInvoice=a.cIDBranchInvoice 
    and e.mc1enabled = 1
    --################ invoice item detail #######################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceItemDetail 
    ) g
    on g.cIDCompany = e.cIDCompany
    and g.cIDInvoice = e.cIDInvoice
    and g.cForm = e.cform
    and g.cSerie = e.cSerie
    AND g.cIDBranchInvoice = e.cIDBranchInvoice
    and g.nSeq = e.nSeq --igualando pasos 
    and g.cidProduct = e.cidProduct --igualando los productos de factura y order
    and g.mc1enabled = 1
    and g.cIDStep in ('00100','00200','00400','00600','00700','00800','00900','01100','01200','01300') --steps para llegar a la venta neta 
    --################### MC1_MC_OrderExt ########################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderExt
    ) f 
    on f.cIDOrder = b.cIDOrder
    and f.cIDCompany = b.cIDCompany
    and f.mc1enabled = 1
    --#################### REQUEST EMITES ######################
    INNER JOIN (SELECT * 
    FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceRequestEmites
    ) m
    on a.cIDCompany = m.cIDCompany
    and a.cIDInvoice = m.cIDInvoice
    and a.cForm = m.cform
    and a.cSerie = m.cSerie
    and a.cIDBranchInvoice =m.cIDBranchInvoice
    and m.mc1enabled = 1
    and UPPER(m.xInvoiceRequestStatus)= 'SUCCEEDED' 
    --################### PRICING_RULE ###################
    inner join (
    select *
    from CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_PricingFormationRule
    ) h
    on g.cIDStep = h.cIDStep
    and g.cIDPricing = h.cIDPricing
    and g.cIDCompany = h.cIDCompany
    and g.mc1enabled = 1
    and h.mc1enabled = 1
    and upper(h.cIDPricingFormationRule) != 'UNITVALUECALCULATED' --se repite el paso 00100 y parece aumentar el valor 
    --############# invoiceext para partial ############
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceExt 
    ) i 
    on a.cIDCompany = i.cIDCompany
    and a.cIDInvoice = i.cIDInvoice
    and a.cForm = i.cform
    and a.cSerie = i.cSerie
    and a.cIDBranchInvoice =i.cIDBranchInvoice
    AND i.mc1Enabled = 1
    ---#################### order para encontrar orden de factura ##########
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_Order
    ) j
    on f.cIDOrderRef = j.cIDOrder --trayendo solo lo que genero otro docto, por rechazo parcial
    and f.cIDCompany = j.cIDCompany
    and f.cidcustomer = j.cidcustomer
    and j.mc1enabled = 1
    --################# order invoice para factura parcial #########################
    left join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderInvoice 
    ) k 
    on j.cIDOrder = k.cIDOrder --trayendo solo lo que genero otro docto, por rechazo parcial
    and j.cIDCompany = k.cIDCompany
    AND k.mc1Enabled = 1
    --################## invoice item por producto #####################
    left join (select *
    FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceItem
    ) l
    on l.cIDCompany = k.cIDCompany 
    and l.cIDInvoice = k.cIDInvoice 
    and l.cForm = k.cform
    and l.cSerie = k.cSerie
    AND l.cIDBranchInvoice=k.cIDBranchInvoice 
    and l.mc1enabled = 1
    and l.cidProduct = e.cidProduct --igualando los productos de factura y remito
    where a.xInvoiceType in ('ZOGRD','ZOGR') --Ordertype no buscados a fin de ver os nuevos
    and format(a.dEmission , 'yyyy-MM-dd') between '2025-01-02' AND '2025-01-07' --informacion de un dia  
    and a.xInvoiceCancelReason!='0'

),
ValidOrders2 AS (
    SELECT DISTINCT
        b.cIDOrder,
        b.cIDProduct,
        a.cIDBranchInvoice
    FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderInvoice a
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderItem b ON
        a.cIDOrder = b.cIDOrder AND
        a.cIDCompany = b.cIDCompany AND
        a.cIDBranchInvoice = b.cIDBranchInvoice
    WHERE
        a.mc1Enabled = 1 AND
        b.mc1Enabled = 1 AND
        CONVERT(DATE, a.mc1LastUpdate) BETWEEN '2025-01-02' AND '2025-01-07'
),
ValidSales2 AS (
    SELECT 
        a.*, 
        b.cIDOrder
    FROM InitialSalesRechazos a
    INNER JOIN ValidOrders2 b ON
        a.Order_DwID = b.cIDOrder AND
        a.Product_DwID = b.cIDProduct AND
        a.BranchDwID = b.cIDBranchInvoice
        AND b.cIDOrder is not null
),
FirstDeduplicates2 AS (
    SELECT
        *
    FROM (
        SELECT
            a.*,
            ROW_NUMBER() OVER (
                PARTITION BY a.BranchDwID, a.ChannelDistribution, a.Company_DwID, a.Customer_DwID, a.Emission, a.Form, a.FreeGoods,
                a.Invoice_DwID, a.InvoiceCancelReason, a.InvoiceType, a.Pricing_DwID, a.Product_DwID, a.Seq, a.Serie, a.Step_DwID, a.Territory_DwID
                ORDER BY a.MC1LastUpdate DESC, a.ReasonReturnStock_DwID DESC
            ) AS rn
        FROM ValidSales2 a
    ) sub
    WHERE rn = 1
),
Transformations23 AS (
    SELECT
        a.*,
        CASE WHEN a.Step_DwID = '00100' THEN [Value] ELSE 0 END AS Valor_Bruto,
        CASE WHEN a.Step_DwID = '00400' THEN [Value] ELSE 0 END AS Promocion_ML,
        CASE WHEN a.Step_DwID = '00600' THEN [Value] ELSE 0 END AS Descuento_ML,
        CASE WHEN a.Step_DwID = '00150' THEN [Value] ELSE 0 END AS Escalafc_ML,
        CASE 
            WHEN a.Step_DwID = '00200' AND a.Freegoods = 0 THEN a.[Value] 
            WHEN a.Step_DwID = '00100' AND a.Freegoods = 1 THEN a.[Value] 
            ELSE 0
        END AS Facdsl_ML,
        CASE WHEN a.Step_DwID = '01300' THEN [Value] ELSE 0 END AS Impuesto1,
        CASE WHEN a.Step_DwID = '01100' THEN [Value] ELSE 0 END AS Impuesto2,
        CASE WHEN a.Step_DwID = '01400' THEN [Value] ELSE 0 END AS Venta_Impuestos
    FROM FirstDeduplicates2 a
    WHERE a.cIDOrder IS NOT NULL
),
GroupedSalesRechazos AS (
    SELECT
        Amount,
        AmountFac,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceFac_DwID,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        SerieFac,
        Territory_DwID,
        Tipo_Rechazo,
        Tipo_Rechazo_Id,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2
    FROM Transformations23
    GROUP BY
        Amount,
        AmountFac,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceFac_DwID,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        SerieFac,
        Territory_DwID,
        Tipo_Rechazo,
        Tipo_Rechazo_Id,
        UnitValue
),
GroupedSalesRechazos2 AS (
      SELECT
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceFac_DwID,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        SerieFac,
        Territory_DwID,
        Tipo_Rechazo,
        Tipo_Rechazo_Id,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Amount) AS Amount,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2,
        SUM(AmountFac) AS AmountFac
    FROM GroupedSalesRechazos
    GROUP BY
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceFac_DwID,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        SerieFac,
        Territory_DwID,
        Tipo_Rechazo,
        Tipo_Rechazo_Id,
        UnitValue
),
AddColumnsRechazos AS (
    SELECT 
        a.BranchDwID codigo_agencia,
        a.ChannelDistribution,
        CASE WHEN a.Company_DwID='OP' THEN 'PERU' ELSE 'REV' END as pais,
        a.Customer_DwID codigo_cliente,
        CONVERT(VARCHAR(10), CAST(a.Emission AS DATE), 120) fecha,
        a.Form,
        a.Invoice_DwID,
        a.InvoiceFac_DwID,
        a.InvoiceCancelReason,
        a.InvoiceType AG19ID,
        a.Order_DwID,
        a.Product_DwID codigo_producto,
        a.ReasonReturnStock_DwID,
        a.Serie,
        a.SerieFac,
        REPLACE(a.Territory_DwID, a.BranchDwID, '') as codigo_ruta,
        a.UnitValue,
        a.Valor_Bruto,
        a.Facdsl_ML,
        a.Promocion_ML,
        a.Descuento_ML,
        a.Amount,
        a.AmountFac,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.Tipo_Rechazo,
        a.Tipo_Rechazo_Id,
        CONCAT(a.Serie,'-',a.Invoice_DwID) as numero,
        CONCAT(a.SerieFac,'-',a.InvoiceFac_DwID) as numero_fac,
        a.ChannelDistribution as cod_linea_ruta,
        'FALSE' as FACCON,
        0 as FACDSL,
        null as dif_precio,
        CASE
            WHEN a.InvoiceType = 'ZBON' THEN 0
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' OR a.InvoiceType = 'ZOGRD' OR a.InvoiceType = 'ZOGR' THEN 0
            ELSE 0
        END AS facdet,
        CASE 
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' OR a.InvoiceType = 'ZOGRD' OR a.InvoiceType = 'ZOGR' 
            THEN CONCAT(a.Tipo_Rechazo_Id,'-',a.InvoiceCancelReason)
            ELSE 'REVISAR'
        END AS Id_Motivo
    FROM GroupedSalesRechazos2 a
),
Subchannel AS (
    SELECT *
    FROM (
        SELECT 
            a.*,
            b.cidbranchinvoice,
            ROW_NUMBER() OVER (PARTITION BY b.cidbranchinvoice, a.cIDCustomer ORDER BY a.mc1Enabled DESC, a.mc1LastUpdate DESC) AS rn
        FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_CustomerDivisionSectorExt a
        LEFT JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_CustomerDivisionSector b 
        ON a.cidcustomer = b.cidcustomer
        WHERE b.cidbranchinvoice IS NOT NULL
    ) sub
    WHERE rn = 1
),
JoinRechazosWithSubchannel AS (
    SELECT
        a.*,
        b.xSubChannelDistribution
    FROM AddColumnsRechazos a
    LEFT JOIN  Subchannel b
    ON a.codigo_cliente = b.cIDCustomer
    AND a.codigo_agencia = b.cidbranchinvoice
),
ModifyColumnsRechazos AS (
    SELECT 
        a.codigo_agencia,
        a.codigo_cliente CLICOD,
        CONVERT(VARCHAR(10), a.fecha, 120) Fecha_vta,
        a.Invoice_DwID no_factura,
        a.InvoiceFac_DwID,
        a.SerieFac,
        a.InvoiceCancelReason,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.UnitValue precio,
        a.dif_precio,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) IN ('ZBON', 'ZOREX', 'ZREX') THEN 0 - a.Facdsl_ML
            ELSE a.Facdsl_ML
        END AS FACDSL$,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) IN ('ZBON', 'ZOREX', 'ZREX') THEN 0 - a.Descuento_ML
            ELSE a.Descuento_ML
        END AS descuento,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) IN ('ZBON', 'ZOREX', 'ZREX') THEN 0 - a.Promocion_ML
            ELSE a.Promocion_ML
        END AS promocion,
        a.Amount cantidad,
        a.AmountFac,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.numero,
        a.numero_fac,
        a.Tipo_Rechazo,
        a.Tipo_Rechazo_Id,
        a.FACCON,
        a.FACDSL,
        a.facdet FACDET,
        CASE  
    		WHEN a.cod_linea_ruta <> a.xSubChannelDistribution AND a.xSubChannelDistribution IS NOT NULL THEN a.xSubChannelDistribution ELSE a.cod_linea_ruta 
		END AS cod_linea_ruta,
        CASE 
            WHEN a.Id_Motivo is null AND a.InvoiceCancelReason <> ''  THEN a.InvoiceCancelReason
            ELSE a.Id_Motivo END as Id_Motivo
    FROM JoinRechazosWithSubchannel a 
),
JoinWithRetunrReasonAndMotivos AS (
    SELECT
        a.codigo_agencia,
        a.CLICOD,
        a.Fecha_vta,
        a.no_factura,
        a.InvoiceFac_DwID,
        a.SerieFac,
        a.InvoiceCancelReason,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.precio,
        a.dif_precio,
        a.FACDSL$ ,
        a.promocion,
        a.descuento,
		a.cantidad - isnull(a.AmountFac,0) AS cantidad,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.numero,
        CASE WHEN a.numero_fac = '-' THEN '' ELSE a.numero_fac END AS numero_fac,
        a.Tipo_Rechazo_Id,
        a.FACCON,
        a.FACDSL,
        a.FACDET,
        a.Id_Motivo,
        a.cod_linea_ruta,
        isnull(a.AmountFac,0) AmountFac,
        CONCAT(a.Tipo_Rechazo, '-', b.Description) AS factura_cliente
    FROM ModifyColumnsRechazos a 
    LEFT JOIN ReturnReason b
    ON a.InvoiceCancelReason = b.cIDDomainType
    LEFT JOIN Motivos c
    ON a.Id_Motivo = c.cIDDomainType
    WHERE b.Description is not null
    AND a.codigo_ruta NOT LIKE '%0A%'
    AND CONCAT(a.Tipo_Rechazo, '-', b.Description) is not null
),
QueryRechazosMC1Chile as (
    SELECT 
        a.codigo_agencia,
        a.CLICOD,
        a.Fecha_vta,
        a.no_factura,
        a.InvoiceCancelReason,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.precio,
        a.dif_precio,
        a.FACDSL$ ,
        a.promocion,
        a.descuento,
        a.cantidad,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.numero,
        a.FACCON,
        a.FACDSL,
        a.FACDET,
        a.Id_Motivo,
        a.factura_cliente,
        a.cod_linea_ruta
    FROM JoinWithRetunrReasonAndMotivos a
    WHERE a.cantidad <> 0
),
FinalUnion AS (
	SELECT
	    codigo_agencia,
	    CLICOD,
	    Fecha_vta,
	    no_factura,
	    InvoiceCancelReason,
	    AG19ID,
	    codigo_producto,
	    codigo_ruta,
	    precio,
	    dif_precio,
	    [FACDSL$],
	    promocion,
	    descuento,
	    cantidad,
	    Impuesto1,
	    Venta_Impuestos,
	    Impuesto2,
	    numero,
	    FACCON,
	    FACDSL,
	    FACDET,
	    Id_Motivo,
	    factura_cliente,
	    cod_linea_ruta
	FROM QueryRechazosMC1Chile
	
	UNION ALL
	
	SELECT
	    codigo_agencia,
	    CLICOD,
	    Fecha_vta,
	    no_factura,
	    InvoiceCancelReason,
	    AG19ID,
	    codigo_producto,
	    codigo_ruta,
	    precio,
	    dif_precio,
	    [FACDSL$],
	    promocion,
	    descuento,
	    cantidad,
	    Impuesto1,
	    Venta_Impuestos,
	    Impuesto2,
	    numero,
	    FACCON,
	    FACDSL,
	    FACDET,
	    Id_Motivo,
	    factura_cliente,
	    cod_linea_ruta
	FROM prejoin
)

SELECT count(*) , sum(facdet) , sum(cantidad) , sum(precio) FROM FinalUnion